# Using ParmEd with MDAnalysis and OpenMM to simulate a selection of atoms
ref: [link](https://userguide.mdanalysis.org/stable/examples/other/parmed_sim.html)


In [1]:
import warnings

import parmed as pmd
import MDAnalysis as mda

from MDAnalysis.tests.datafiles import PRM7_ala2, RST7_ala2

# suppress some MDAnalysis warnings when writing PDB files
warnings.filterwarnings('ignore')
home_path = "/data/home/bty391/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook"

## Loading files: the difference between ParmEd and MDAnalysis

Both ParmEd and MDAnalysis read a number of file formats. However, while MDAnalysis is typically used to analyse simulations, ParmEd is often used to set them up. This requires ParmEd to read topology parameter information that MDAnalysis typically ignores, such as the equilibrium length and force constants of bonds in the system. For example, the ParmEd structure below.

In [2]:
pprm = pmd.load_file(PRM7_ala2, RST7_ala2)
print("pprm: ")
print(pprm.bonds[0])
pprm


pprm: 
<Bond <Atom C [10]; In ALA 0>--<Atom O [11]; In ALA 0>; type=<BondType; k=570.000, req=1.229>>


<AmberParm 3026 atoms; 1003 residues; 3025 bonds; PBC (orthogonal); parameterized>

In [3]:
# When MDAnalysis reads these files in, 
# it does not include that information.
mprm = mda.Universe(PRM7_ala2, RST7_ala2, format='RESTRT')
print("MDAnalysis: ")
print(mprm.atoms.bonds[0].type)
print(mprm)

# If you then convert this Universe to ParmEd, 
# you can see that the resulting Structure is not parametrized.
mprm_converted = mprm.atoms.convert_to('PARMED')
print(mprm_converted)

# While the bonds are present, there is no type information associated.
print(mprm_converted.bonds[0])

MDAnalysis: 
('N3', 'H')
<Universe with 3026 atoms>
<Structure 3026 atoms; 1003 residues; 3025 bonds; parameterized>
<Bond <Atom N [0]; In ALA 0>--<Atom H1 [1]; In ALA 0>; type=None>


Therefore, if you wish to use ParmEd functionality that requires parametrization on a MDAnalysis Universe, you need to create that Universe from a ParmEd structure in order to convert it back to something useable in ParmEd.

In [4]:
mprm_from_parmed = mda.Universe(pprm)
mprm_from_parmed

<Universe with 3026 atoms>

In [5]:
mprm_from_parmed.bonds[0].type

<Bond <Atom N [0]; In ALA 0>--<Atom H1 [1]; In ALA 0>; type=<BondType; k=434.000, req=1.010>>

## Using MDAnalysis to select atoms
One reason we might want to convert a ParmEd structure into MDAnalysis is to use its sophisticated atom selection syntax. While ParmEd has its own ways to select atoms, MDAnalysis allows you to select atoms based on geometric distance.

In [6]:
water = mprm_from_parmed.select_atoms('around 5 protein').residues.atoms
protein_shell = mprm_from_parmed.select_atoms('protein') + water
prm_protein_shell = protein_shell.convert_to('PARMED')

In [7]:
prm_protein_shell

<Structure 155 atoms; 46 residues; 154 bonds; PBC (orthogonal); parameterized>

## Using ParmEd and OpenMM to create a simulation system

In [8]:
import sys
import openmm as mm
import openmm.app as app

from parmed import unit as u
from parmed.openmm import StateDataReporter, MdcrdReporter

In [9]:
# You can create an OpenMM simulation system directly from a ParmEd structure, 
# providing that it is parametrized.
system = prm_protein_shell.createSystem(nonbondedMethod=app.NoCutoff,
                                        constraints=app.HBonds,
                                        implicitSolvent=app.GBn2)

In [10]:
# Here we set the integrator to do Langevin dynamics.
integrator = mm.LangevinIntegrator(
                        300*u.kelvin,       # Temperature of heat bath
                        1.0/u.picoseconds,  # Friction coefficient
                        2.0*u.femtoseconds, # Time step
                        )

In [11]:
# We create the Simulation object and set particle positions.
sim = app.Simulation(prm_protein_shell.topology, system, integrator)
sim.context.setPositions(prm_protein_shell.positions)

In [12]:
# We now minimise the energy.
sim.minimizeEnergy(maxIterations=500)

In [13]:
# The reporter below reports energies and coordinates every 100 steps to stdout, 
# but every 10 steps to the ala2_shell.trj file.

sim.reporters.append(
        StateDataReporter(sys.stdout, 100, step=True, potentialEnergy=True,
                          kineticEnergy=True, temperature=True, volume=True,
                          density=True)
)
sim.reporters.append(MdcrdReporter(f'{home_path}/output/ala2_shell.trj', 10, crds=True))

In [14]:
sim.step(500)

#"Step","Time (ps)","Potential Energy (kilocalorie/mole)","Kinetic Energy (kilocalorie/mole)","Total Energy (kilocalorie/mole)","Temperature (K)","Box Volume (angstrom**3)","Density (gram/(item*milliliter))"
100,0.20000000000000015,-619.358213230611,21.84976611869639,-597.5084471119146,69.15238436042476,45325.8064191062,0.034909350700361955
200,0.4000000000000003,-609.3382492081217,41.66032046719123,-567.6779287409305,131.8508618296167,45325.8064191062,0.034909350700361955
300,0.6000000000000004,-600.5994495359992,54.77432720122843,-545.8251223347706,173.35541749629516,45325.8064191062,0.034909350700361955
400,0.8000000000000006,-598.1994333556703,68.07572574488663,-530.1237076107836,215.45305001214948,45325.8064191062,0.034909350700361955
500,1.0000000000000007,-600.3997281640394,72.95647189804465,-527.4432562659947,230.90013681918143,45325.8064191062,0.034909350700361955


In [15]:
# If we write a topology file out from our former protein_shell atomgroup, 
# we can load the trajectory in for further analysis.

protein_shell.write(f'{home_path}/output/ala2_shell.pdb')
u = mda.Universe(f'{home_path}/output/ala2_shell.pdb', f'{home_path}/output/ala2_shell.trj')
u.trajectory

<TRJReader /data/home/bty391/Project/QMUL_HPC_note_w_MDAnalysis/Tutorial_notebook/output/ala2_shell.trj with 50 frames of 155 atoms>